In [1]:
import json
import ast
import csv
import ast
import pickle
import string
import requests
from Bio import SeqIO
from time import sleep
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict
import pandas as pd
import os
import functools
import operator as op
import numpy as np
from scipy.stats import binom
import nferx_py.fn as nf

product = lambda iterable: functools.reduce(op.mul, iterable, 1)
nf.authenticate("nikhilkayal@nference.net", "063a1c676867563c7898102c1dfd9b0e", "preview")

nferx_py v1.7.6 is currently installed. Please upgrade to the latest version v2.0.0. Installation link: https://github.com/lumenbiomics/nferx_py


2020-06-08 09:41:06,968 : INFO : URL: https://preview.nferx.com/api/is_authenticated, process-time: 0.5146870613098145
2020-06-08 09:41:06,969 : INFO : Authentication successful - nferX is online
2020-06-08 09:41:07,403 : INFO : URL: https://preview.nferx.com/api/get_data_versions, process-time: 0.432873010635376
2020-06-08 09:41:07,404 : INFO : Data version "202003" has been set as default one


# Parsing human & corona proteome

In [2]:
coronavirus = "/Users/nikhil/Downloads/37975558536-AllCoronaviridae_ProteinFastaResults.fasta"
human = "/Users/nikhil/nference/repositories/nfer/nferxapps/bio2vec_api/common/resources/uniprot-proteome-tabs.txt"

In [3]:
def parse_coronavirus_fasta(file):
    print("Reading: ", file)
    corona_family = {}
    for record in SeqIO.parse(file, "fasta"):
        corona_family[str(record.description)] = str(record.seq)
    corona_id_to_seq = {}
    corona_id_to_organism = {}
    corona_id_to_protein = {}
    for key, value in corona_family.items():
        corona_id = key.split("|")[1].split(":")[1]
        corona_organism = key.split("|")[3].split(":")[1]
        corona_id_to_seq[corona_id] = value
        corona_id_to_organism[corona_id] = corona_organism
        if len(key.split("|")) > 5:
            corona_protein = key.split("|")[5].split(":")[1]
        corona_id_to_protein[corona_id] = corona_protein
    organism_to_keys = defaultdict(list)
    for key, value in sorted(corona_id_to_organism.items()):
        organism_to_keys[value].append(key)
    organism_to_keys = dict(organism_to_keys)
    return corona_id_to_seq, corona_id_to_protein, organism_to_keys

In [4]:
corona_id_to_seq, corona_id_to_protein, organism_to_keys = parse_coronavirus_fasta(coronavirus)

Reading:  /Users/nikhil/Downloads/37975558536-AllCoronaviridae_ProteinFastaResults.fasta


In [5]:
def read_uniprot_exome(uniprot_exome_file):
    fd = open(uniprot_exome_file, 'rU')
    # delim = '\t' if (uniprot_exome_file.endswith('.tsv')) else ','
    delim = '\t'
    csv_file = csv.reader(fd, delimiter=delim)
    next(csv_file)  # skip header

    print("Reading: ", uniprot_exome_file)

    seq_dict = dict()
    protein_dict = dict()
    uniprot_dict = dict()
    uniprot2protein = dict()
    uniprot2gene = dict()

    for row in csv_file:
        uniprot_id = row[0]
        protein = row[1]
        gene = row[2]
        seq = row[3]

        if len(gene) == 0:
            if len(protein) > 0:
                gene = protein
            else:
                gene = ['uniprot_' + uniprot_id]
                pass
        gene_str = gene.split()[0]
        gene_str = gene_str.upper()
        seq_dict[gene_str] = seq
        protein_dict[gene_str] = protein
        uniprot_dict[uniprot_id] = seq
        uniprot2protein[uniprot_id] = protein
        uniprot2gene[uniprot_id] = gene_str

    return seq_dict, protein_dict, uniprot_dict, uniprot2protein, uniprot2gene

In [6]:
seq_dict, protein_dict, uniprot_dict, uniprot2protein, uniprot2gene = read_uniprot_exome(human)

Reading:  /Users/nikhil/nference/repositories/nfer/nferxapps/bio2vec_api/common/resources/uniprot-proteome-tabs.txt


# Surprise Score Calculation

In [7]:
# def count_nmers(sequence_dict, n):
#     counts = defaultdict(int)
#     for sequence in sequence_dict.values():
#         for start_ix in range(len(sequence) - n + 1):
#             counts[sequence[start_ix:start_ix + n]] += 1
#     series = pd.Series(counts)
#     return series

In [8]:
# def compute_probs(sequence_dict, n):
#     symbol_counts = count_nmers(sequence_dict, 1)
#     symbol_freqs = (symbol_counts / symbol_counts.sum()).to_dict()
#     counts = count_nmers(sequence_dict, n)
#     probs = {nmer: product(symbol_freqs[symbol] for symbol in nmer) for nmer in list(counts.index)}
#     series = pd.Series(probs)
#     return series

In [9]:
# def computer_nmer_suprise(sequence_dict, n, db):
# #     counts = self.count_nmers(n)
#     counts = pd.read_csv(db + "_{0}.csv".format(n), index_col=0, header=None, keep_default_na=False)[1]
# #     probs = self.compute_probs(n)
#     probs = pd.read_csv(db + "_prob_{0}.csv".format(n), index_col=0, header=None, keep_default_na=False)[1]
#     total_length_n_sequences = sum(max(0, len(seq) - n + 1) for seq in sequence_dict.values())
#     expected = probs * total_length_n_sequences
#     surprise_ratios = counts / expected
#     log2_surprise_ratio = np.log2(surprise_ratios)
#     variances = probs * (1 - probs) * total_length_n_sequences
#     stds = np.sqrt(variances)
#     stds_from_mean = (counts - expected) / stds
#     percentiles = np.array([binom.cdf(count, n=total_length_n_sequences, p=prob)
#                                     for count, prob in zip(counts, probs)])
#     survivals = np.array([binom.sf(count, n=total_length_n_sequences, p=prob)
#                                   for count, prob in zip(counts, probs)])  # 1 - percentiles, but more accurate
#     uniqueness_percentile_score = -np.log10(percentiles * 100)
#     uniqueness_percentile_score[np.isinf(uniqueness_percentile_score)] = 500
#     conservation_percentile_score = -np.log10(survivals * 100)
#     conservation_percentile_score[np.isinf(conservation_percentile_score)] = 500
#     df = pd.DataFrame(dict(
#                     count=counts,
#                     probability=probs,
#                     expected=expected,
#                     surprise_ratio=surprise_ratios,
#                     log2_surprise_ratio=log2_surprise_ratio,
#                     std=stds,
#                     stds_from_mean=stds_from_mean,
#                     percentile=percentiles,
#                     uniqueness_percentile_score=uniqueness_percentile_score,
#                     conservation_percentile_score=conservation_percentile_score
#                 ))
#     return df

In [10]:
# uniprot_8_surprise = computer_nmer_suprise(uniprot_dict, 8, "uniprot")
# uniprot_8_surprise.to_csv("uniprot_8_surprise.csv")

# uniprot_9_surprise = computer_nmer_suprise(uniprot_dict, 9, "uniprot")
# uniprot_9_surprise.to_csv("uniprot_9_surprise.csv")

# uniprot_10_surprise = computer_nmer_suprise(uniprot_dict, 10, "uniprot")
# uniprot_10_surprise.to_csv("uniprot_10_surprise.csv")

# viral_8_surprise = computer_nmer_suprise(corona_id_to_seq, 8, "viral")
# viral_8_surprise.to_csv("coronavirus/corona_8_surprise.csv")

# viral_9_surprise = computer_nmer_suprise(corona_id_to_seq, 9, "viral")
# viral_9_surprise.to_csv("coronavirus/corona_9_surprise.csv")

# viral_10_surprise = computer_nmer_suprise(corona_id_to_seq, 10, "viral")
# viral_10_surprise.to_csv("coronavirus/corona_10_surprise.csv")

# Find longest matching string

In [11]:
def longest_substring(s1, s2):
    t = [[0]*(1+len(s2)) for i in range(1+len(s1))]
    l, xl = 0, 0
    for x in range(1,1+len(s1)):
        for y in range(1,1+len(s2)):
            if s1[x-1] == s2[y-1]:
                t[x][y] = t[x-1][y-1] + 1
                if t[x][y]>l:
                    l = t[x][y]
                    xl  = x
            else:
                t[x][y] = 0
    return s1[xl-l: xl]

In [12]:
def find_viral_species(key_to_find):
    for key, value in organism_to_keys.items():
        if key_to_find in value:
            return key

In [ ]:
temp_l = []
for corona_id, corona_seq in corona_id_to_seq.items():
    print(corona_id)
    for human_id, human_seq in uniprot_dict.items():
        match = longest_substring(corona_seq, human_seq)
        if match:
            temp_dict = dict()
            temp_dict["peptide"] = match
            temp_dict["peptide_length"] = len(match)
            temp_dict["human_protein"] = uniprot2gene[human_id]
            temp_dict["human_protein_annotation"] = human_id
            temp_dict["viral_species"] = find_viral_species(corona_id)
            temp_dict["viral_protein"] = corona_id_to_protein[corona_id]
            temp_dict["viral_protein_annotation"] = corona_id
            temp_dict["human_protein_peptide_start"] = uniprot_dict[human_id].find(match)
            temp_dict["human_protein_peptide_end"] = temp_dict["human_protein_peptide_start"] + temp_dict["peptide_length"] - 1
            temp_dict["viral_protein_peptide_start"] = corona_id_to_seq[corona_id].find(match)
            temp_dict["viral_protein_peptide_end"] = temp_dict["viral_protein_peptide_start"] + temp_dict["peptide_length"] - 1
            temp_l.append(temp_dict)

ALT05254.1
ALT05263.1
ALT05264.1
ALT05255.1
ALT05256.1
ALT05257.1
ALT05258.1
ALT05259.1
ALT05260.1
ALT05261.1
ALT05262.1
AOC31971.1
AOC31972.1
AOC31973.1
AOC31974.1
AOC31975.1
AOC31976.1
AOC31977.1
AOC31978.1
AOC31982.1
AOC31979.1
AOC31980.1
AOC31981.1
AZF86122.1
AZF86119.1
AZF86118.1
AZF86120.1
AZF86121.1
AZF86134.1
AZF86130.1
AZF86131.1
AZF86132.1
AZF86133.1
AZF86128.1
AZF86127.1
AZF86126.1
AZF86125.1
AZF86124.1
YP_009755891.1
YP_009755890.1
YP_009755894.1
YP_009755892.1
YP_009755893.1
QBG64639.1
VIPR_ALG1_AVY53335_1_11507_11842.1
VIPR_ALG1_AVY53335_1_8966_9787.1
VIPR_ALG1_AVY53335_1_11873_12229.1
VIPR_ALG1_AVY53335_1_9485_10495.1
VIPR_ALG1_AVY53335_1_9815_10099.1
AVY53336.1
AVY53337.1
AVY53338.1
AVY53339.1
AVY53340.1
AVY53341.1
AVY53342.1
VIPR_ALG1_AVY53334_1_12286_13332.1
VIPR_ALG1_AVY53334_1_15838_16275.1
VIPR_ALG1_AVY53334_1_16084_16803.1
VIPR_ALG1_AVY53334_1_16312_16773.1
VIPR_ALG1_AVY53334_1_16837_18579.1
VIPR_ALG1_AVY53334_1_19396_20289.1
VIPR_ALG1_AVY53335_1_10673_10921.1
VIP

In [15]:
len(temp_l)

156794